In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv


def get_table_headers(table):
    """Given a table soup, returns all the headers"""
    headers = []
    for th in table.find("tr").find_all("th"):
        headers.append(th.text.strip())
    return headers

def get_table_rows(table):
    """Given a table, returns all its rows"""
    rows = []
    for tr in table.find_all("tr")[1:]:
        cells = []
        # grab all td tags in this table row
        tds = tr.find_all("td")
        if len(tds) == 0:
            # if no td tags, search for th tags
            # can be found especially in wikipedia tables below the table
            ths = tr.find_all("th")
            for th in ths:
                cells.append(th.text.strip())
        else:
            # use regular td tags
            for td in tds:
                cells.append(td.text.strip())
        rows.append(cells)
    return rows

def save_as_csv(table_name, headers, rows):
    pd.DataFrame(rows, columns=headers).to_csv(f"{table_name}.csv")
    
def main(url):
    
    source = requests.get(url).text
    soup = BeautifulSoup(source,'lxml')
    table = soup.find('table')
    table_name = 'output'
    
     # get the table headers
    headers = get_table_headers(table)
    # get all the rows of the table
    rows = get_table_rows(table)
    
    save_as_csv(table_name, headers, rows)
    
if __name__ == "__main__":
    import sys
    try:
        url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
    except IndexError:
        print("Please specify a URL.\nUsage: python html_table_extractor.py [URL]")
        exit(1)
    main(url)

In [2]:
df= pd.read_csv('output.csv',na_values ='Not assigned')
df.head()

,Unnamed: 0,Postal code,Borough,Neighborhood
0,0,M1A,NaN,NaN
1,1,M2A,NaN,NaN
2,2,M3A,North York,Parkwoods
3,3,M4A,North York,Victoria Village
4,4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [3]:
df.drop('Unnamed: 0',axis = 1, inplace = True)

In [4]:
df.dropna(axis=0, subset=['Borough'],inplace = True)

In [5]:
df['Neighborhood'] = df['Neighborhood'].str.replace(r'/', ',')

In [6]:
df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [7]:
mask = df['Neighborhood'] == "Not assigned"
df.loc[mask, 'Neighborhood'] = df.loc[mask, 'Borough']

In [8]:
df1 = pd.read_csv("http://cocl.us/Geospatial_data")
df1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
df1.drop("Postal Code",axis=1,inplace = True)

In [10]:
df1.head()

,Latitude,Longitude
0,43.806686,-79.194353
1,43.784535,-79.160497
2,43.763573,-79.188711
3,43.770992,-79.216917
4,43.773136,-79.239476
